In [40]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text

In [41]:
import re 

def preprocess_minutes(minutes):

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)
    #print(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)


    return sections, section_texts

In [42]:
import pandas as pd
from ekonlpy.sentiment import MPCK

def text2ngram(text) :
    mpck = MPCK()
    tokens = mpck.tokenize(text)
    ngrams = mpck.ngramize(tokens)

    return tokens+ ngrams

In [43]:
import csv
import pandas as pd
from os import listdir
from os.path import isfile, join

   
def preprocessing(source_folder="./K_bank_minutes_txt/", output_file="./section_minutes.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=['date', 'minutes'])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    df["ngram"] = ""
    

    for txt_file in txt_files :
        try :
            with open(source_folder + txt_file, 'r',encoding='utf-8') as f :
                txt = f.read()
                sections, section_texts = preprocess_minutes(txt)

                print(txt_file)

                ngrams = []
                for section_text in section_texts[1:3] :
                    for sentence in section_text :
                        ngrams.append(','.join(text2ngram(sentence)))

                #print('@@@'.join(ngrams))
                #break

                df.loc[len(df)] = [txt_file.split('_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),
                                   '@@@'.join(ngrams)]
        except :
            pass
    
    df.to_csv(output_file, index=False)
        
    print(df.shape)
preprocessing()

2005.10.11_금융통화위원회 의사록2005년도 제21차.txt
2005.11.10_금융통화위원회 의사록2005년도 제24차.txt
2005.12.22_금융통화위원회 의사록2005년도 제27차.txt
2005.12.8_금융통화위원회 의사록2005년도 제26차.txt
2005.7.7_금융통화위원회 의사록2005년도 제14차.txt
2005.8.11_금융통화위원회 의사록2005년도 제17차.txt
2005.9.22_금융통화위원회 의사록2005년도 제20차.txt
2005.9.8_금융통화위원회 의사록2005년도 제19차.txt
2006.1.12_금융통화위원회 의사록2006년도 제2차.txt
2006.1.26_금융통화위원회 의사록2006년도 제3차.txt
2006.1.5_금융통화위원회 의사록2006년도 제1차.txt
2006.10.12_금융통화위원회 의사록2006년도 제21차.txt
2006.10.26_금융통화위원회 의사록2006년도 제22차.txt
2006.11.23_금융통화위원회 의사록2006년도 제24차.txt
2006.11.9_금융통화위원회 의사록2006년도 제23차.txt
2006.12.21_금융통화위원회 의사록2006년도 제26차.txt
2006.12.7_금융통화위원회 의사록2006년도 제25차.txt
2006.2.9_금융통화위원회 의사록2006년도 제4차.txt
2006.3.23_금융통화위원회 의사록2006년도 제7차.txt
2006.3.9_금융통화위원회 의사록2006년도 제6차.txt
2006.4.7_금융통화위원회 의사록2006년도 제8차.txt
2006.5.11_금융통화위원회 의사록2006년도 제10차.txt
2006.5.25_금융통화위원회 의사록2006년도 제11차.txt
2006.6.22_금융통화위원회 의사록2006년도 제13차.txt
2006.6.8_금융통화위원회 의사록2006년도 제12차.txt
2006.7.7_금융통화위원회 의사록2006년도 제14차.txt
2006.8.10_금융통화위원회 의사록2006년도 제16차.txt
2006.8.17

2016.1.28._금융통화위원회 의사록2016년도 제2차.txt
2016.10.13._금융통화위원회 의사록2016년도 제20차.txt
2016.10.27._금융통화위원회 의사록2016년도 제21차.txt
2016.11.11._금융통화위원회 의사록2016년도 제22차.txt
2016.11.24._금융통화위원회 의사록2016년도 제23차.txt
2016.12.15._금융통화위원회 의사록2016년도 제24차.txt
2016.12.21._금융통화위원회 의사록2016년도 제25차.txt
2016.12.29._금융통화위원회 의사록2016년도 제26차.txt
2016.2.16_금융통화위원회 의사록2016년도 제3차.txt
2016.2.25._금융통화위원회 의사록2016년도 제4차.txt
2016.3.10._금융통화위원회 의사록2016년도 제5차.txt
2016.3.24._금융통화위원회 의사록2016년도 제6차.txt
2016.4.19._금융통화위원회 의사록2016년도 제7차.txt
2016.5.13._금융통화위원회 의사록2016년도 제9차.txt
2016.6.23._금융통화위원회 의사록2016년도 제12차.txt
2016.6.9._금융통화위원회 의사록2016년도 제11차.txt
2016.7.1._금융통화위원회 의사록2016년도 제13차.txt
2016.7.14._금융통화위원회 의사록2016년도 제14차.txt
2016.8.11._금융통화위원회 의사록2016년도 제16차.txt
2016.9.22._금융통화위원회 의사록2016년도 제19차.txt
2016.9.9._금융통화위원회 의사록2016년도 제18차.txt
2017.1.13._금융통화위원회 의사록2017년도 제1차.txt
2017.1.26._금융통화위원회 의사록2017년도 제2차.txt
2017.10.19._금융통화위원회 의사록2017년도 제19차.txt
2017.11.30._금융통화위원회 의사록2017년도 제22차.txt
2017.11.9._금융통화위원회 의사록2017년도 제21차.txt
2017.12.14._금융통화

In [44]:
import pandas as pd
df = pd.read_csv("./section_minutes.csv")
df['date'] = pd.to_datetime(df['date'])
df.head(2)

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,2005-10-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
1,2005-11-10,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 향후 미국의 안정실업률 (NAIRU: Non Accelerating ...,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...,NaN,NaN,NaN,14,8,157,0,0,0,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V..."


In [45]:
df = df.rename(columns = {'minutes':'content'})
df.head(2)

,date,content,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count,ngram
0,2005-10-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 지방자치단체의 투자유치활동 이 활발해지고 항공기, 철도, 자동차 등의...",일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,NaN,NaN,NaN,11,4,158,0,0,0,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
1,2005-11-10,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 향후 미국의 안정실업률 (NAIRU: Non Accelerating ...,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...,NaN,NaN,NaN,14,8,157,0,0,0,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V..."


In [46]:
df.drop(['Economic Situation'],axis='columns',inplace=True)
df.drop(['Monetary Policy'],axis='columns',inplace=True)
df.drop(['Participant Views'],axis='columns',inplace=True)
df.drop(['Government View'],axis='columns',inplace=True)
df.drop(['Economic Situation count'],axis='columns',inplace=True)
df.drop(['Foreign Currency count'],axis='columns',inplace=True)
df.drop(['Financial Markets count'],axis='columns',inplace=True)
df.drop(['Monetary Policy count'],axis='columns',inplace=True)
df.drop(['Participant Views count'],axis='columns',inplace=True)
df.drop(['Government View count'],axis='columns',inplace=True)

In [47]:
df.head(2)

,date,content,Foreign Currency,Financial Markets,ngram
0,2005-10-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,일부 위원은 미국과 우리나라의 정책금 리 격차가 확대되고 있는 가운데 금년 두 차...,일부 위원은 8월말 이후 우리나라 주가지수가 큰 폭 상승한 것은 경제성장에 대한...,"금리갭/NNG,확대/NNG,되/XSV,가운데/NNG,남/VV,fed/NNG,금리/N..."
1,2005-11-10,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"일부 위원은 미 달러화에 대한 엔화 약 세가 지속될 것으로 보는지 물었으며, 이에...",일부 위원은 장단기금리 격차가 경제성장률에 선행하는 모습을 보인다고 하면 미국의...,"미/NNG,달러/NNG,엔/NNG,약세/NNG,지속/NNG,되/VV,보/VV,물/V..."


In [7]:
import pandas as pd
df = pd.read_csv("04_K_bank_minute_ngram.csv",encoding = 'utf-8')

In [3]:
df = df.sort_values('date')

In [8]:
A = df['Foreign Currency']+df['Financial Markets']

In [11]:
df['section']=""

In [13]:
df['section']=A

In [15]:
del df['Foreign Currency']
del df['Financial Markets']


In [27]:
df = df.dropna()

In [28]:
len(df)

125

In [6]:
df.to_csv("04_K_bank_minute_ngram.csv",encoding = 'utf-8')

In [5]:
del df['Unnamed: 0']
del df['Unnamed: 0.1']

In [1]:
import pandas as pd 
df = pd.read_csv("04_K_bank_minute_ngram.csv",encoding = 'utf-8')

In [7]:
df.tail(4)

,Unnamed: 0,date,content,ngram,section
121,224,2016-10-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"금융시장/NNG,체로/NNG,안정/NNG,보/VV,예외적/VAX,물가/NNG,변동성...",일부 위원은 최근 주요국의 금융시장이 대 체로 안정적인 모습을 보이고 있지만 영국...
122,226,2016-11-11,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"대선/NNG,결과/NNG,경제/NNG,오/VV,국제/NNG,유가/NNG,미치/VV,...",일부 위원은 미국의 대선 결과가 중국경제 와 국제유가에 미치는 영향에 대해 물었으...
123,228,2016-12-15,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"재정/NNG,지출/NNG,확대/NNG,보호무역주의/NNG,강화/NNG,fed/NNG...","일부 위원은 미국의 재정지출 확대, 보호무 역주의 강화 등으로 미 연준의 금리인상..."
124,244,2017-01-13,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"임금/NNG,상승률/NNG,확대/NNG,되/XSV,물가/NNG,상승/NNG,압력/N...",일부 위원은 미국의 시간당평균임금 상승률이 확대되고 있어 물가상승 압력 이 높아질...


In [12]:
df['section'][124]

'일부 위원은 미국의 시간당평균임금 상승률이 확대되고 있어 물가상승 압력  이 높아질 가능성이 있으며 미 연준도 조만간 물가상승률이 목표수준에 도달할  것으로 보여 추가적인 확장적 재정정책이 필요하지 않다는 입장인 것으로 보인다  고 언급하면서, 그럼에도 불구하고 앞으로 이러한 정책이 추진될 경우에는 미 연  준의 금리인상 속도가 시장의 예상보다 빨라질 가능성이 있다는 의견을 나타내었  음.@@@이에 대해 관련부서에서는 앞으로 미국의 물가상승 압력이 재정지출 확대 등  으로 증대되면서 미 연준의 금리인상 속도가 예상보다 빨라질 것이라는 관측도  제기되고 있지만, 전반적으로 시장참가자들은 미국경제의 회복 속도가 그만큼 빠  르지 않을 수 있고 재정지출을 확대하는 데에도 많은 어려움이 수반될 수 있다는  점에서 기존의 전망을 유지하고 있는 것으로 알고 있다고 답변하였음.@@@이어서 동 위원은 원화 환율의 변동성이 높아질 경우에는 수출 중소기업이  환위험 관리에 어려움을 겪는 등 다양한 여파가 예상된다고 언급하면서, 이에 대  한 관련부서의 견해를 물었으며,  이에 대해 관련부서에서는 원화 환율의 변동성 확대는 경제주체들의 소비나  투자에 부정적 영향을 미치기 때문에 동 변동성이 완화될 수 있도록 지속적으로  노력하고 있다고 답변하였음.@@@이에 동 위원은 한편으로는 원화 환율이 신축적으로 조정되지 못할 경우에도  이에 따른 압력이 외환시장에 누적될 우려가 있다고 덧붙였음.@@@- 11 -  다른 일부 위원은 금년 들어 원/달러 환율의 변동성이 확대되고 있는 이유가  무엇인지 관련부서에 물었으며,  이에 대해 관련부서에서는 미국 금리인상에 대한 기대가 FOMC 의사록이나  고용지표의 발표 등에 따라 달라지면서 원/달러 환율의 변동성이 높아지고 있다  고 답변하고, 여타 신흥국 통화보다 원화의 NDF 거래가 활성화되고 있는 점도  일부 영향을 미치고 있는 것으로 보인다고 첨언하였음.@@@이어서 동 위원은 최근 중국이 위안화 약세를 저지하기 위해 외환시장 개입  및 자